#### Get cleansed_weather_table

In [0]:
%run "/Users/075bei015.kshitiz@pcampus.edu.np/DB6- APIToCleansed"

In [0]:
# display(df_cleansed_data)

#### Fact table for hourly data

In [0]:
# %sql

# drop table fact_weatherData_hourly

In [0]:
def create_fact_table():
    spark.sql("""
        create table fact_weather_data_hourly(
          dt INT,
          date_alt_key DATE,
          time_alt_key TIMESTAMP,
          id INT,
          name VARCHAR(100),
          lat FLOAT,
          lon FLOAT,
          temp FLOAT,
          temp_min FLOAT,
          temp_max FLOAT,
          pressure INT,
          humidity INT,
          visibility INT,
          wind_speed FLOAT,
          wind_deg FLOAT,
          wind_gust FLOAT,
          clouds_all INT
        )
   """)

In [0]:
# create_fact_table()

In [0]:
def upsert_into_facthourly():
    spark.sql("""
            insert into fact_weather_data_hourly
            select
              cw.dt,
              dd.date_alt_key,
              dt.time_alt_key,
              dc.id,
              dc.name,
              cw.lat,
              cw.lon,
              cw.temp,
              cw.temp_min,
              cw.temp_max,
              cw.pressure,
              cw.humidity,
              cw.visibility,
              cw.wind_speed,
              cw.wind_deg,
              cw.wind_gust,
              cw.clouds_all
            from
              cleansed_weather as cw
              inner join dim_date as dd on cw.date = dd.date_alt_key
              inner join dim_time as dt on hour(cw.time) = hour(dt.time_alt_key)
              inner join dim_city as dc on cw.city_name = dc.name
            where
              (cw.date, hour(cw.time)) not in (select distinct date_alt_key, hour(time_alt_key) from fact_weather_data_hourly)
       """)

In [0]:
upsert_into_facthourly()

In [0]:
# df_fact_weather_hourly = spark.read.table("fact_weather_data_hourly")

In [0]:
@keep_log
def get_fact_with_log(df):
    df = df.select(
            col('dt'),
            col('date_alt_key'),
            col('time_alt_key'),
            col('id').alias('city_id'),
            col('name').alias('city_name'),
            col('lat'),
            col('lon'),
            col('temp'),
            col('temp_min'),
            col('temp_max'),
            col('pressure'),
            col('humidity'),
            col('visibility'),
            col('wind_speed'),
            col('wind_deg'),
            col('wind_gust'),
            col('clouds_all'))
    
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return df, start, end

In [0]:
# df_fact_hourly = get_fact_with_log('FACT', 'fact_weather_hourly', df_fact_weather_hourly)

In [0]:
# %sql

# select * from fact_weather_hourly

In [0]:
# %sql

# select * from weather_log

#### Fact table for daily data

In [0]:
def create_fact_table_daily():
    spark.sql("""
        create table fact_weather_data_daily(
          dt INT,
          date_alt_key DATE,
          id INT,
          name VARCHAR(100),
          lat FLOAT,
          lon FLOAT,
          temp FLOAT,
          temp_min FLOAT,
          temp_max FLOAT,
          pressure INT,
          humidity INT,
          visibility INT,
          wind_speed FLOAT,
          wind_deg FLOAT,
          wind_gust FLOAT,
          clouds_all INT
        )
   """)

In [0]:
# create_fact_table_daily()

In [0]:
def insert_into_factdaily():
    spark.sql("""
            insert into fact_weather_data_daily
            select
              dt,
              date_alt_key,
              id,
              name,
              lat,
              lon,
              avg(temp) over (partition by name,date_alt_key),
              temp_min,
              temp_max,
              avg(pressure) over (partition by name,date_alt_key),
              avg(humidity) over (partition by name,date_alt_key),
              avg(visibility) over (partition by name,date_alt_key),
              avg(wind_speed) over (partition by name,date_alt_key),
              avg(wind_deg) over (partition by name,date_alt_key),
              avg(wind_gust) over (partition by name,date_alt_key),
              avg(clouds_all) over (partition by name,date_alt_key)
            from
                fact_weather_data_hourly
       """)

In [0]:
insert_into_factdaily()

In [0]:
# df_fact_weather_daily = spark.read.table("fact_weather_data_daily")

In [0]:
@keep_log
def get_fact_daily_with_log(df):
    df = df.select(
            col('dt'),
            col('date_alt_key'),
            col('id').alias('city_id'),
            col('name').alias('city_name'),
            col('lat'),
            col('lon'),
            col('temp'),
            col('temp_min'),
            col('temp_max'),
            col('pressure'),
            col('humidity'),
            col('visibility'),
            col('wind_speed'),
            col('wind_deg'),
            col('wind_gust'),
            col('clouds_all'))
    
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return df, start, end

In [0]:
# df_fact_daily = get_fact_daily_with_log('FACT', 'fact_weather_daily', df_fact_weather_daily)

In [0]:
# %sql

# select * from fact_weather_daily

In [0]:
# %sql

# select * from weather_log